In [3]:
import socket
from datetime import datetime
from random import getrandbits
from ipaddress import IPv4Address
from netaddr import IPNetwork, IPAddress
import threading
import requests
from urllib.request import urlopen
import json

ModuleNotFoundError: No module named 'netaddr'

In [2]:
print('''Don't Mess With Network List from Mirai:
127.0.0.0/8               # Loopback
0.0.0.0/8                 # Invalid address space
3.0.0.0/8                 # General Electric (GE)
15.0.0.0/7                # Hewlett-Packard (HP)
56.0.0.0/8                # US Postal Service
10.0.0.0/8                # Internal network
192.168.0.0/16            # Internal network
172.16.0.0/14             # Internal network
100.64.0.0/10             # IANA NAT reserved
169.254.0.0/16            # IANA NAT reserved
198.18.0.0/15             # IANA Special use
224.0.0.0/4               # Multicast
6.0.0.0/7                 # Department of Defense 
11.0.0.0/8                # Department of Defense
21.0.0.0/8                # Department of Defense
22.0.0.0/8                # Department of Defense
26.0.0.0/8                # Department of Defense
28.0.0.0/7                # Department of Defense
30.0.0.0/8                # Department of Defense
33.0.0.0/8                # Department of Defense
55.0.0.0/8                # Department of Defense
214.0.0.0/7               # Department of Defense''')

Don't Mess With Network List from Mirai:
127.0.0.0/8               # Loopback
0.0.0.0/8                 # Invalid address space
3.0.0.0/8                 # General Electric (GE)
15.0.0.0/7                # Hewlett-Packard (HP)
56.0.0.0/8                # US Postal Service
10.0.0.0/8                # Internal network
192.168.0.0/16            # Internal network
172.16.0.0/14             # Internal network
100.64.0.0/10             # IANA NAT reserved
169.254.0.0/16            # IANA NAT reserved
198.18.0.0/15             # IANA Special use
224.0.0.0/4               # Multicast
6.0.0.0/7                 # Department of Defense 
11.0.0.0/8                # Department of Defense
21.0.0.0/8                # Department of Defense
22.0.0.0/8                # Department of Defense
26.0.0.0/8                # Department of Defense
28.0.0.0/7                # Department of Defense
30.0.0.0/8                # Department of Defense
33.0.0.0/8                # Department of Defense
55.0.0.0/8      

In [3]:
dontMessWithList = ['127.0.0.0/8', '0.0.0.0/8', '3.0.0.0/8', '15.0.0.0/7', '56.0.0.0/8', '10.0.0.0/8', '192.168.0.0/16', '172.16.0.0/14', '100.64.0.0/10', '169.254.0.0/16', '198.18.0.0/15', '224.0.0.0/4', '6.0.0.0/7', '11.0.0.0/8', '21.0.0.0/8', '22.0.0.0/8', '26.0.0.0/8', '28.0.0.0/7', '30.0.0.0/8', '33.0.0.0/8', '55.0.0.0/8', '214.0.0.0/7']

In [4]:
def generate_rand_ip():
    ip_in_bits = getrandbits(32)
    ip_addr = IPv4Address(ip_in_bits)
    rand_ip_addr = str(ip_addr)
    return rand_ip_addr

In [5]:
def check_ip(ip_addr):
    for network in dontMessWithList:
        if IPAddress(ip_addr) in IPNetwork(network):
            return False
    return True

In [6]:
print('Some standard ports: \nTR-069:\t7547\nUPnP:\t1900\nXMPP:\t5222\nCoAP:\t5683\nMQTT:\t1883/8883')

Some standard ports: 
TR-069:	7547
UPnP:	1900
XMPP:	5222
CoAP:	5683
MQTT:	1883/8883


In [7]:
def TCP_connect(ip, port_number, delay, output):
    TCPsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    TCPsock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    TCPsock.settimeout(delay)
    try:
        TCPsock.connect((ip, port_number))
        output[port_number] = 'Listening'
    except:
        output[port_number] = ''


def scan_ports(host_ip, delay):

#     print("-" * 60)
#     print("Please wait, scanning remote host", host_ip)
#     print("-" * 60)

    t1 = datetime.now()

    threads = []
    output = {}
    common_iot_ports = [21, 22, 23, 25, 80, 81, 82, 83, 84, 88, 137, 143, 443, 445, 554, 631, 1080, 1883, 1900, 2000, 2323, 4433, 4443, 4567, 5222, 5683, 7474, 7547, 8000, 8023, 8080, 8081, 8443, 8088, 8883, 8888, 9000, 9090, 9999, 10000, 37777, 49152]

    for i in common_iot_ports:
        t = threading.Thread(target=TCP_connect, args=(host_ip, i, delay, output))
        threads.append(t)

    for i in range(len(common_iot_ports)):
        threads[i].start()
    
    for i in range(len(common_iot_ports)):
        threads[i].join()
    
    total_listening = 0
    ports_listening = []
    for i in range(len(common_iot_ports)):
        if output[common_iot_ports[i]] == 'Listening':
            total_listening += 1
            ports_listening.append(common_iot_ports[i])
#             print(str(common_iot_ports[i]) + ': ' + output[common_iot_ports[i]])

#     t2 = datetime.now()
#     total =  t2 - t1
#     print('Scanning completed in:', total)
    if total_listening > 0:
        print("IP:", host_ip)
        print(total_listening, ports_listening)
    return total_listening, ports_listening

In [8]:
def check_if_phue_bulb(ip_addr, port):
    url = 'http://' + ip_addr + ':' + str(port)
    try:
        r = requests.get(url, verify=False, timeout=2)
        return r.headers
        r = urlopen(url, timeout=3, verify=False)
        string = r.read().decode('utf-8')
        json_obj = json.loads(string)
        return json_obj
    except Exception as e:
        return "Empty json object"

In [9]:
import pickle

def storeData(obj, filename):
    pickleFile = open(filename, 'wb')
    pickle.dump(obj, pickleFile)
    pickleFile.close()
    
def loadData(filename):
    pickleFile = open(filename, 'rb')
    obj = pickle.load(pickleFile)
    pickleFile.close()
    return obj

In [12]:
# Every one of them is a set
ips_checked = loadData("../addresses/ips_checked.dat")
hikvision_camera_addr = loadData("../addresses/hikvision_camera_addr.dat")
sonicWall_firewall_addr = loadData("../addresses/sonicWall_firewall_addr.dat")
netgear_router_addr = loadData("../addresses/netgear_router_addr.dat")
TR069_protocolDevice_addr = loadData("../addresses/TR069_protocolDevice_addr.dat")
lighttpd_protocolDevice_addr = loadData("../addresses/lighttpd_protocolDevice_addr.dat")
Huawei_router_addr = loadData("../addresses/Huawei_router_addr.dat")
kangle_addr = loadData("../addresses/kangle_addr.dat")
tplink_router_addr = loadData("../addresses/tplink_router_addr.dat")
app_web_server_addr = loadData("../addresses/app_web_server_addr.dat")

In [13]:
try:
    while True:
        ip_addr = generate_rand_ip()
        if ip_addr not in ips_checked:
            ips_checked.add(ip_addr)
        else:
            continue
    #     ip_addr = '73.162.12.235'
        if check_ip(ip_addr):
            a, b = scan_ports(ip_addr, 2)
            if a > 0:
                for port in b:
    #                 print("Reading port:", port)
                    json_obj = check_if_phue_bulb(ip_addr, port)
#                     print(json_obj)
                    if json_obj != "Empty json object":
                        print(json_obj)
                        rh = json.dumps(json_obj.__dict__['_store'])
                        print(rh)
                        if 'Hikvision'.lower() in rh.lower() or 'DVRDVS'.lower() in rh.lower():
                            hikvision_camera_addr.add(ip_addr + ":" + str(port))
                        elif 'SonicWALL'.lower() in rh.lower():
                            sonicWall_firewall_addr.add(ip_addr + ":" + str(port))
                        elif 'NETGEAR'.lower() in rh.lower():
                            netgear_router_addr.add(ip_addr + ":" + str(port))
                        elif 'TR069'.lower() in rh.lower() or 'gSOAP'.lower() in rh.lower() or 'TR-069'.lower() in rh.lower():
                            TR069_protocolDevice_addr.add(ip_addr + ":" + str(port))
                        elif 'lighttpd'.lower() in rh.lower():
                            lighttpd_protocolDevice_addr.add(ip_addr + ":" + str(port))
                        elif 'HuaweiHomeGateway'.lower() in rh.lower():
                            Huawei_router_addr.add(ip_addr + ":" + str(port))
                        elif 'kangle'.lower() in rh.lower():
                            kangle_addr.add(ip_addr + ":" + str(port))
                        elif 'TP-LINK'.lower() in rh.lower():
                            tplink_router_addr.add(ip_addr + ":" + str(port))
                        elif 'App-webs'.lower() in rh.lower():
                            app_web_server_addr.add(ip_addr + ":" + str(port))
                    if 'name' in json_obj:
                        if json_obj['name'] == 'Philips hue':
                            print(("*" * 10) + 'Philips hue bulb is found.' + ("*" * 10))
    #                 print()
    #     else:
    #         print(ip_addr, "is in excluded ip list.")
    #     break
except KeyboardInterrupt:
    pass

IP: 118.150.197.32
1 [7547]
{'Connection': 'close', 'Content-Type': 'text/html', 'Content-Length': '89'}
{"connection": ["Connection", "close"], "content-type": ["Content-Type", "text/html"], "content-length": ["Content-Length", "89"]}
IP: 172.80.30.169
1 [80]
{'Server': 'squid', 'Date': 'Tue, 26 Mar 2024 14:18:25 GMT', 'Content-Type': 'text/html', 'Last-Modified': 'Fri, 16 Jun 2023 20:28:13 GMT', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'ETag': 'W/"648cc5dd-a0"', 'Content-Encoding': 'gzip'}
{"server": ["Server", "squid"], "date": ["Date", "Tue, 26 Mar 2024 14:18:25 GMT"], "content-type": ["Content-Type", "text/html"], "last-modified": ["Last-Modified", "Fri, 16 Jun 2023 20:28:13 GMT"], "transfer-encoding": ["Transfer-Encoding", "chunked"], "connection": ["Connection", "keep-alive"], "vary": ["Vary", "Accept-Encoding"], "etag": ["ETag", "W/\"648cc5dd-a0\""], "content-encoding": ["Content-Encoding", "gzip"]}
IP: 154.201.134.26
2 [80, 443]
{'

C:\Users\Brunda\AppData\Roaming\Python\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '31.14.23.13'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Brunda\AppData\Roaming\Python\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mimp.ro'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'Connection': 'Keep-Alive', 'Keep-Alive': 'timeout=5, max=100', 'content-type': 'text/html; charset=UTF-8', 'link': '<https://mimp.ro/>; rel=shortlink', 'etag': '"5118-1710951904;gz"', 'x-litespeed-cache': 'hit', 'content-encoding': 'gzip', 'vary': 'Accept-Encoding', 'content-length': '26825', 'date': 'Tue, 26 Mar 2024 14:19:51 GMT', 'content-security-policy': 'upgrade-insecure-requests;', 'alt-svc': 'h3=":443"; ma=2592000, h3-29=":443"; ma=2592000, h3-Q050=":443"; ma=2592000, h3-Q046=":443"; ma=2592000, h3-Q043=":443"; ma=2592000, quic=":443"; ma=2592000; v="43,46"'}
{"connection": ["Connection", "Keep-Alive"], "keep-alive": ["Keep-Alive", "timeout=5, max=100"], "content-type": ["content-type", "text/html; charset=UTF-8"], "link": ["link", "<https://mimp.ro/>; rel=shortlink"], "etag": ["etag", "\"5118-1710951904;gz\""], "x-litespeed-cache": ["x-litespeed-cache", "hit"], "content-encoding": ["content-encoding", "gzip"], "vary": ["vary", "Accept-Encoding"], "content-length": ["content-

C:\Users\Brunda\AppData\Roaming\Python\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '31.14.23.13'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Brunda\AppData\Roaming\Python\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mimp.ro'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'Connection': 'Keep-Alive', 'Keep-Alive': 'timeout=5, max=100', 'content-type': 'text/html; charset=UTF-8', 'link': '<https://mimp.ro/>; rel=shortlink', 'etag': '"5118-1710951904;gz"', 'x-litespeed-cache': 'hit', 'content-encoding': 'gzip', 'vary': 'Accept-Encoding', 'content-length': '26825', 'date': 'Tue, 26 Mar 2024 14:19:53 GMT', 'content-security-policy': 'upgrade-insecure-requests;', 'alt-svc': 'h3=":443"; ma=2592000, h3-29=":443"; ma=2592000, h3-Q050=":443"; ma=2592000, h3-Q046=":443"; ma=2592000, h3-Q043=":443"; ma=2592000, quic=":443"; ma=2592000; v="43,46"'}
{"connection": ["Connection", "Keep-Alive"], "keep-alive": ["Keep-Alive", "timeout=5, max=100"], "content-type": ["content-type", "text/html; charset=UTF-8"], "link": ["link", "<https://mimp.ro/>; rel=shortlink"], "etag": ["etag", "\"5118-1710951904;gz\""], "x-litespeed-cache": ["x-litespeed-cache", "hit"], "content-encoding": ["content-encoding", "gzip"], "vary": ["vary", "Accept-Encoding"], "content-length": ["content-

In [14]:
print("Total unique IPs checked:", len(ips_checked))
print("Total unique hikvision cameras found:", len(hikvision_camera_addr))
print("Total unique sonicWall firewall found:", len(sonicWall_firewall_addr))
print("Total unique netgear router found:", len(netgear_router_addr))
print("Total unique TR069_protocolDevice found:", len(TR069_protocolDevice_addr))
print("Total unique lighttpd_protocolDevice found:", len(lighttpd_protocolDevice_addr))
print("Total unique Huawei_router found:", len(Huawei_router_addr))
print("Total unique kangle found:", len(kangle_addr))
print("Total unique tplink_router found:", len(tplink_router_addr))
print("Total unique app_web_server found:", len(app_web_server_addr))

Total unique IPs checked: 174684
Total unique hikvision cameras found: 2
Total unique sonicWall firewall found: 15
Total unique netgear router found: 1
Total unique TR069_protocolDevice found: 310
Total unique lighttpd_protocolDevice found: 43
Total unique Huawei_router found: 103
Total unique kangle found: 0
Total unique tplink_router found: 4
Total unique app_web_server found: 10


In [15]:
print(hikvision_camera_addr)
print(sonicWall_firewall_addr)
print(netgear_router_addr)
print(TR069_protocolDevice_addr)
print(lighttpd_protocolDevice_addr)
print(Huawei_router_addr)
print(kangle_addr)
print(tplink_router_addr)
print(app_web_server_addr)

{'65.35.209.134:80', '58.162.228.234:80'}
{'68.132.199.74:80', '23.31.252.105:443', '75.147.201.173:80', '66.195.139.130:80', '207.109.58.115:80', '109.247.79.6:80', '86.62.172.158:80', '75.148.90.253:80', '207.109.58.115:443', '91.84.53.230:80', '98.112.144.122:80', '173.8.228.205:80', '216.23.194.170:80', '195.101.63.225:80', '193.248.210.66:80'}
{'2.27.7.110:7547'}
{'75.135.49.190:7547', '66.67.42.213:7547', '186.114.231.87:7547', '81.130.154.121:7547', '47.41.96.77:7547', '86.156.106.39:7547', '109.152.60.163:7547', '86.172.106.63:7547', '31.52.26.254:7547', '81.158.163.155:7547', '174.110.37.250:7547', '86.184.115.84:7547', '95.9.76.86:7547', '47.33.211.34:7547', '67.248.134.108:7547', '75.139.19.81:7547', '79.52.240.29:7547', '81.152.227.54:7547', '24.74.97.90:7547', '217.42.56.245:7547', '61.69.204.207:7547', '75.191.198.101:7547', '183.83.190.235:7547', '68.112.218.87:7547', '86.136.236.195:7547', '186.217.38.13:81', '47.40.191.44:7547', '172.248.39.105:7547', '87.58.102.192:75

In [16]:
storeData(ips_checked, "ips_checked.dat")
storeData(hikvision_camera_addr, "hikvision_camera_addr.dat")
storeData(sonicWall_firewall_addr, "sonicWall_firewall_addr.dat")
storeData(netgear_router_addr, "netgear_router_addr.dat")
storeData(TR069_protocolDevice_addr, "TR069_protocolDevice_addr.dat")
storeData(lighttpd_protocolDevice_addr, "lighttpd_protocolDevice_addr.dat")
storeData(Huawei_router_addr, "Huawei_router_addr.dat")
storeData(kangle_addr, "kangle_addr.dat")
storeData(tplink_router_addr, "tplink_router_addr.dat")
storeData(app_web_server_addr, "app_web_server_addr.dat")

In [18]:
from collections import defaultdict


port_dict = defaultdict(lambda : 0)


for i in hikvision_camera_addr:
    port_dict[i.split(':')[1]] += 1
for i in sonicWall_firewall_addr:
    port_dict[i.split(':')[1]] += 1
for i in netgear_router_addr:
    port_dict[i.split(':')[1]] += 1
for i in TR069_protocolDevice_addr:
    port_dict[i.split(':')[1]] += 1
for i in lighttpd_protocolDevice_addr:
    port_dict[i.split(':')[1]] += 1
for i in Huawei_router_addr:
    port_dict[i.split(':')[1]] += 1
for i in kangle_addr:
    port_dict[i.split(':')[1]] += 1
for i in tplink_router_addr:
    port_dict[i.split(':')[1]] += 1
for i in app_web_server_addr:
    port_dict[i.split(':')[1]] += 1

print(port_dict)

defaultdict(<function <lambda> at 0x000002351A2077F0>, {'80': 38, '443': 2, '7547': 392, '81': 6, '82': 2, '9090': 1, '10000': 1, '8000': 3, '8080': 7, '8088': 6, '83': 1, '88': 2, '8081': 25, '1080': 1, '9999': 1})
